In [1]:
import pandas as pd

In [15]:
data = pd.read_csv("rating_final.csv")
data = data[["userID", "placeID", "rating"]]

In [16]:
data.head()

,userID,placeID,rating
0,1077,135085,2
1,1077,135038,2
2,1077,132825,2
3,1077,135060,1
4,1068,135104,1


In [25]:
# Create the utility matrix
umatrix = data.pivot(columns="userID", values="rating")

In [36]:
undefined_values = umatrix.isna().sum().sum()
total_values = umatrix.shape[0] * umatrix.shape[1]

sparsity = int(undefined_values / total_values * 10000)
print(f"{sparsity/100.0}% empty")

99.27% empty


## Hybrid recommender
![alt text](feature_augment_graph.jpeg "Feature augment")
Source: https://medium.com/analytics-vidhya/7-types-of-hybrid-recommendation-system-3e4f78266ad8